# AI-Book-Recommender: Visualization
This notebook generates visualizations to gain insights into user behavior, rating patterns, and model performance, including heatmaps, PCA plots, and distribution charts.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.sparse.linalg import svds

# Set seaborn style
sns.set(style='whitegrid')

# Load cleaned datasets
books = pd.read_csv('books.csv', low_memory=False)
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
# Filter to top users and books
top_users = ratings['User-ID'].value_counts().head(1000).index
top_books = ratings['ISBN'].value_counts().head(1000).index
ratings_small = ratings[ratings['User-ID'].isin(top_users) & ratings['ISBN'].isin(top_books)]

In [ ]:
# SVD Visualization with PCA
def svd_visualize(ratings):
    user_item_matrix = ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
    user_ratings_mean = user_item_matrix.mean(axis=1)
    R_demeaned = user_item_matrix.sub(user_ratings_mean, axis=0)
    U, sigma, Vt = svds(R_demeaned, k=50)
    pca = PCA(n_components=2)
    user_pca = pca.fit_transform(U)
    plt.figure(figsize=(8, 6))
    plt.scatter(user_pca[:, 0], user_pca[:, 1], alpha=0.5)
    plt.title('User Visualization in SVD-Reduced Space via PCA')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True)
    plt.show()

svd_visualize(ratings_small)

In [ ]:
# User Age Distribution
plt.figure(figsize=(12, 6))
sns.boxplot(x=users['Age'], color='lightgreen')
plt.title('User Age Distribution (Boxplot)')
plt.xlabel('Age')
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of Top 50 Users vs Top 50 Books
top_50_users = ratings['User-ID'].value_counts().head(50).index
top_50_books = ratings['ISBN'].value_counts().head(50).index
ratings_heatmap = ratings[ratings['User-ID'].isin(top_50_users) & ratings['ISBN'].isin(top_50_books)]
heatmap_matrix = ratings_heatmap.pivot(index='User-ID', columns='ISBN', values='Book-Rating')

plt.figure(figsize=(15, 10))
sns.heatmap(heatmap_matrix, cmap='YlGnBu', cbar_kws={'label': 'Rating'})
plt.title('Heatmap of Ratings: Top 50 Users vs Top 50 Books')
plt.xlabel('Book ISBN')
plt.ylabel('User ID')
plt.tight_layout()
plt.show()